In [10]:
import pandas as pd
import numpy as np
import tweepy
import datetime
import pytz
import time
from datetime import timezone

In [3]:
BT = "AAAAAAAAAAAAAAAAAAAAAEdLhgEAAAAAKyu0W6PIo7Zj%2FNB1BpINf%2Fcm6gE%3DT0EVYBss7PGh2rxZ70CXaOvKMrCifCwfVwsK37dXrcPSA851gg"
CK = "zScfmXxDONhXg9cIjp2SEk6m6"
CS = "fuAXsoZjTU5zvE1MkievSa7oITjF414wU6EGQUSgBwuTSfT44h"
AT = "1557281594955923456-ae2iV205fcP82YzSkCj3T9fPCQEEjh"
AS = "yOYPCy5G7ty8L6OikJWewpbSdEvGGbl1ETHgrRdw8ffGU"

api = tweepy.Client(bearer_token=BT, consumer_key=CK, consumer_secret=CS, access_token=AT, access_token_secret=AS, wait_on_rate_limit=True)

def utc2jst(timestamp_utc):
    datetime_utc = datetime.datetime.strptime(timestamp_utc + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_jst = datetime_utc.astimezone(datetime.timezone(datetime.timedelta(hours=+9)))
    timestamp_jst = datetime.datetime.strftime(datetime_jst, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_jst

def jst2utc(timestamp_jst):
    datatime_jst = datetime.datetime.strptime(timestamp_jst + ".000+0000", "%Y-%m-%dT%H:%M:%SZ.%f%z")
    datetime_utc = datatime_jst.astimezone(datetime.timezone(datetime.timedelta(hours=-9)))
    timestamp_utc = datetime.datetime.strftime(datetime_utc, '%Y-%m-%dT%H:%M:%SZ')
    return timestamp_utc

def twittertime2jst(time):
    utc = datetime.datetime(time.year, time.month,time.day, \
    time.hour,time.minute,time.second, tzinfo=timezone.utc)
    jst = utc.astimezone(pytz.timezone("Asia/Tokyo"))
    str_time = jst.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time


def make_timestamp(yyyy,mm,dd,hh):
    start_time =(str(yyyy)+ '-'+str(mm) +'-' +str(dd) +'T' +str(hh)+':00:00Z')
    end_time = (str(yyyy)+ '-'+str(mm) +'-' +str(dd) +'T' +str(hh)+':59:59Z')
    start_time = jst2utc(start_time)
    end_time = jst2utc(end_time)
    return start_time,end_time

def clean_text_forCSV(text):
    if text ==None:
        return text
    else:
        text = text.replace("\n", "")
        text = text.replace(",", "")
        return text


In [6]:
start_time,end_time = make_timestamp(str(2020),str(10),'01','00')
search_word = '(あ OR い ORう OR え OR お OR か OR き OR く OR け OR こ OR さ OR し OR す OR せ OR そ OR た OR ち OR つ OR て OR と OR な OR に OR ぬ OR ね OR の OR は OR ひ OR ふ OR へ OR ほ OR ま OR み OR む OR め OR も OR や OR ゆ OR よ OR ら OR り OR る OR れ OR ろ OR わ OR を OR ん)lang:ja'


In [11]:
list_tweets = []
counter = 0
for response in tweepy.Paginator(api.search_all_tweets,
                                query = search_word,
                                user_fields = ['username', 'public_metrics', 'description', 'location',],
                                tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 'source', 'attachments'],
                                media_fields = ['url', 'type', ],
                                expansions = ['author_id', 'attachments.media_keys'],
                                start_time=start_time,
                                end_time=end_time,
                                max_results=100,limit=1):
    time.sleep(1)
    list_tweets.append(response)
    #break
result = []
user_dict = {}
# Loop through each response object
for response in list_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': clean_text_forCSV(user.username),
                            'followers': user.public_metrics['followers_count'],
                            'tweets': user.public_metrics['tweet_count'],
                            'description': clean_text_forCSV(user.description),
                            'location': clean_text_forCSV(user.location)
                            }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        created_at = twittertime2jst(tweet.created_at)
        if tweet.geo != None:
            result.append({'text': clean_text_forCSV(tweet.text),
                        'created_at': created_at,
                        'retweets': tweet.public_metrics['retweet_count'],
                        'replies': tweet.public_metrics['reply_count'],
                        'likes': tweet.public_metrics['like_count'],
                        'quote_count': tweet.public_metrics['quote_count'],
                        'author_id': tweet.author_id,
                        'username': author_info['username'],
                        'author_followers': author_info['followers'],
                        'author_tweets': author_info['tweets'],
                        'author_description': author_info['description'],
                        'author_location': author_info['location']
                        })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [12]:
list_tweets

[Response(data=[<Tweet id=1311334987048734720 text='＃錦織\nあーあ。\n今日はもう寝るか'>, <Tweet id=1311334987019350016 text='@10toride01 お誕生日おめでとうございますーーーーー！！！！！！！✨🌹🎉🎊🎂🍗🥓🥩🍣🍕🍷🍾まだまだ大変なご時世ですがいっぱい美味しい物食べていっぱい推しを摂取してください〜！！！'>, <Tweet id=1311334986998448133 text='@ayashama69 めっちゃ可愛いじゃないですか!\n\nでもアイアンメイデン聴いてるみたいな（笑）'>, <Tweet id=1311334986994216961 text='すみませんかのんくん引けた方いましたらアルバム埋めさせてもらえませんか！？！？後でTL遡ってお声がけするかもしれない\u3000すみません……😭😭😭'>, <Tweet id=1311334986994208769 text='RT @smapsmap563: #ミッドナイトスワン\n登場時の一果ちゃんと同じ年の息子に是非観て欲しいと思った\n男の子だから感想を多くは言わなかったけれど\n上映中ずっと前のめりで真剣に観ていたのが伝わって来た\n鑑賞直後の息子の表情を見て一緒に観れて良かったと心から思ったよ\n#…'>, <Tweet id=1311334986985873409 text='Estonia🇪🇪\n絵本の中に迷い込んだみたいな可愛い可愛い町Tallinn🕊💗城壁に囲まれてるから初めてには心強かった〜〜 https://t.co/c7I2MGnICG'>, <Tweet id=1311334986969047042 text='RT @nhk_news: 東京 順天堂大学 練馬病院でクラスターか 新たに33人感染確認 #nhk_news https://t.co/15fISvIXBQ'>, <Tweet id=1311334986964824064 text='RT @cheeky531: 写真を投稿しましたhttps://t.co/cf5j5xJWEw'>, <Tweet id=1311334986939871234 text='@wef_neushia @7g845qq2RiW66Hb 1